In [1]:
import torch
import timm
import backbone
import anchor
import importlib
import Custom_dataset
import cv2
import albumentations as A
import matplotlib.pyplot as plt
import math
from torch import optim
from albumentations.pytorch import ToTensorV2
from torch.optim import lr_scheduler 

from CustomAugment import Cutmix
from configuration import dataset_config
from configuration import dataloader_config
from configuration import OptimizerConfig
from configuration import augmentation_config
from configuration import training_config




c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(Custom_dataset)


image_dir = dataset_config.image_dir
label_dir = dataset_config.label_dir
classes = dataset_config.classes
img_size = dataset_config.image_size

image_test =Custom_dataset.ListDataset(image_dir,
                                       label_dir,
                                       classes,
                                       transform=augmentation_config.transform)

# RabdinSized BBox Safe Crop has box bigger than 1% of image size
train_transform = A.Compose([
    A.RandomSizedBBoxSafeCrop(width=img_size,height=img_size,erosion_rate=0.8),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.2),
], bbox_params=A.BboxParams(format='yolo', # Specify input format
                           label_fields=['class_labels'], # Specify label argument name(s)
                            ))

final_transform = A.Compose([
    A.AdditiveNoise(noise_type="gaussian",
                    spatial_mode="constant",
                    noise_params={"mean_range": (0.0, 0.0), "std_range": (0.05, 0.15)}),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1.0),
    A.Affine(translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)},  # x축 10~20%, y축 -20~20% 랜덤 이동
             p=0.3),
    A.Affine(rotate = (-20,20), p = 0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', # Specify input format
                           label_fields=['class_labels'], # Specify label argument name(s)
                           ))

aug_para ={"alpha":augmentation_config.alpha,
           "lambda":augmentation_config.lamda,
           "prob":augmentation_config.prob}


image_test.add_agumentation(train_transform,aug_para,final_transform)

set_size = len(image_test)
train_dataset_size = int(0.8*set_size)
test_dataset_size = set_size-train_dataset_size

train_indces, valid_indces = torch.utils.data.random_split(range(set_size), [train_dataset_size, test_dataset_size])
print("set size: {0}, test_set size : {1}".format(train_dataset_size,test_dataset_size))


train_dataset = torch.utils.data.Subset(image_test, train_indces)
valid_dataset = torch.utils.data.Subset(image_test, train_indces)
# This problem is occured in Dater type interupt between array and tensor

batch_size = 32
num_workers = 1

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=dataloader_config.batch_size,
                                               num_workers=dataloader_config.num_workers,
                                               shuffle = dataloader_config.shuffle,
                                               collate_fn = image_test.collate_fn)

valid_dataloader = torch.utils.data.DataLoader(valid_dataset,
                                               batch_size=dataloader_config.batch_size,
                                               num_workers=dataloader_config.num_workers,
                                               collate_fn = image_test.collate_fn)

  

for i, data in enumerate(train_dataset):
    print(f"\n🌀 step : {i}")
    print(f"bboxes shape: {data['bboxes'].shape}")
    print(f"class_labels shape: {data['class_labels'].shape}")
    
    result_bboxes, result_labels = image_test.encoder.encoder(data['bboxes'], data['class_labels'])
    
    print(f"result_bboxes shape: {result_bboxes.shape}")
    print(f"result_labels shape: {result_labels.shape}")
    
    print(f"ignore label: {(result_labels == -1).sum().item()}")
    print(f"background label: {(result_labels == 0).sum().item()}") 
    print(f"0 label: {(result_labels == 1).sum().item()}") 
    print(f"1 label: {(result_labels == 2).sum().item()}") 




set size: 1084, test_set size : 272

🌀 step : 0
bboxes shape: torch.Size([5, 4])
class_labels shape: torch.Size([5])
result_bboxes shape: torch.Size([10647, 4])
result_labels shape: torch.Size([10647])
ignore label: 518
background label: 8518
0 label: 907
1 label: 704

🌀 step : 1
bboxes shape: torch.Size([3, 4])
class_labels shape: torch.Size([3])
result_bboxes shape: torch.Size([10647, 4])
result_labels shape: torch.Size([10647])
ignore label: 744
background label: 9005
0 label: 233
1 label: 665

🌀 step : 2
bboxes shape: torch.Size([1, 4])
class_labels shape: torch.Size([1])
result_bboxes shape: torch.Size([10647, 4])
result_labels shape: torch.Size([10647])
ignore label: 453
background label: 10177
0 label: 0
1 label: 17

🌀 step : 3
bboxes shape: torch.Size([1, 4])
class_labels shape: torch.Size([1])
result_bboxes shape: torch.Size([10647, 4])
result_labels shape: torch.Size([10647])
ignore label: 498
background label: 10045
0 label: 0
1 label: 104

🌀 step : 4
bboxes shape: torch.Siz

In [3]:
#Temporary code for testing
model = timm.create_model('resnet50', pretrained=True, num_classes=0)  # Load a pre-trained ResNet50 model without the final classification layer

# Define total batch size 
total_batch_size = math.ceil(train_dataset_size / dataloader_config.batch_size)  # total batch size for all GPUs, accumulate, and gradient steps

nbs = 64  # nominal batch size
accumulate = max(round(nbs / total_batch_size), 1)  # accumulate loss before optimizing
modified_weight_decay= OptimizerConfig.weight_decay * total_batch_size * accumulate / nbs  # scale weight_decay
optimizer = optim.Adam(params = model.parameters(),
                       lr=OptimizerConfig.lr0, 
                       weight_decay=modified_weight_decay, #L2 regularization
                       betas=(0.9,0.999) )  # adjust beta1 to momentum

# Scheduler https://arxiv.org/pdf/1812.01187.pdf
# https://pytorch.org/docs/stable/_modules/torch/optim/lr_scheduler.html#OneCycleLR
warmup_end_lr =  (1-OptimizerConfig.lrf)/2+OptimizerConfig.lrf  # warmup end learning rate
warmup_scheduler = lr_scheduler.LinearLR(optimizer, 
                                         start_factor=OptimizerConfig.lr0, 
                                         end_factor= warmup_end_lr,
                                         total_iters=training_config.warmup_epochs)

# Cosine decay after warmup
epochs_cosine = training_config.epochs - training_config.warmup_epochs
lf = lambda x: ((1 + math.cos(x * math.pi / epochs_cosine)) / 2) * (1 - OptimizerConfig.lrf) + OptimizerConfig.lrf  # cosine
cosine_scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)


In [5]:
model = backbone.YoloV4Model(num_classes=len(dataset_config.classes),
                             to_vector = True)

TypeError: YoloV4Model.__init__() got an unexpected keyword argument 'to_vector'